# Data Gold

**Capa:** Gold

---


### Step 1
**Objetivo:** Genera tablas de negocio agregadas listas para análisis o visualización.

**Tablas:**
- airbnb.gold.listing_score
- airbnb.gold.host_performance
- airbnb.gold.city_market
- airbnb.gold.amenity_popularity

---


In [0]:
-- Elimina datos previos
DROP TABLE IF EXISTS airbnb.gold.listing_score;
DROP TABLE IF EXISTS airbnb.gold.host_performance;
DROP TABLE IF EXISTS airbnb.gold.city_market;
DROP TABLE IF EXISTS airbnb.gold.amenity_popularity;

-- -------------------------------------------
-- GOLD.LISTING_SCORE
-- -------------------------------------------
INSERT OVERWRITE TABLE airbnb.gold.listing_score
WITH rr AS (
  SELECT listing_sk, COUNT(*) AS total_reviews
  FROM airbnb.silver.reviews
  GROUP BY listing_sk
)
SELECT
  l.listing_sk,
  l.city,
  l.country,
  l.review_scores_rating                                                      AS avg_rating,
  COALESCE(rr.total_reviews, 0)                                               AS total_reviews,
  l.price                                                                     AS price,
  CASE
    WHEN months_between(COALESCE(l.last_review_date, current_date()), l.first_review_date) <= 0
      THEN NULL
    ELSE rr.total_reviews / months_between(COALESCE(l.last_review_date, current_date()), l.first_review_date)
  END                                                                         AS reviews_per_month,
  DATEDIFF(current_date(), COALESCE(l.last_review_date, current_date()))      AS review_recency_days,
  current_timestamp()                                                         AS ingest_ts
FROM airbnb.silver.listings l
LEFT JOIN rr ON rr.listing_sk = l.listing_sk;


-- -------------------------------------------
-- GOLD.HOST_PERFORMANCE
-- -------------------------------------------
INSERT OVERWRITE TABLE airbnb.gold.host_performance
WITH li AS (
  SELECT
    l.host_sk,
    h.host_name,
    l.country,
    COUNT(DISTINCT l.listing_sk)             AS total_listings,
    AVG(l.review_scores_rating)              AS avg_rating,
    AVG(CAST(l.price AS DOUBLE))             AS avg_price
  FROM airbnb.silver.listings l
  LEFT JOIN airbnb.silver.hosts h ON h.host_sk = l.host_sk
  GROUP BY l.host_sk, h.host_name, l.country
),
rr AS (
  SELECT l.host_sk, COUNT(*) AS total_reviews
  FROM airbnb.silver.reviews r
  JOIN airbnb.silver.listings l ON l.listing_sk = r.listing_sk
  GROUP BY l.host_sk
),
sh AS (
  SELECT host_sk,
         AVG(CASE WHEN host_is_superhost THEN 1.0 ELSE 0.0 END) AS superhost_rate
  FROM airbnb.silver.hosts
  GROUP BY host_sk
)
SELECT
  li.host_sk,
  li.host_name,
  li.country,
  li.total_listings,
  li.avg_rating,
  li.avg_price,
  COALESCE(rr.total_reviews, 0) AS total_reviews,
  COALESCE(sh.superhost_rate, 0) AS superhost_rate,
  current_timestamp() AS ingest_ts
FROM li
LEFT JOIN rr USING (host_sk)
LEFT JOIN sh USING (host_sk);


-- -------------------------------------------
-- GOLD.CITY_MARKET
-- -------------------------------------------
INSERT OVERWRITE TABLE airbnb.gold.city_market
WITH base AS (
  SELECT
    city, country,
    COUNT(*)                           AS total_listings,
    AVG(CAST(price AS DOUBLE))         AS avg_price,
    AVG(review_scores_rating)          AS avg_rating
  FROM airbnb.silver.listings
  GROUP BY city, country
),
rev AS (
  SELECT l.city, l.country,
         COUNT(*) AS total_reviews,
         COUNT(DISTINCT l.host_sk) AS active_hosts
  FROM airbnb.silver.reviews r
  JOIN airbnb.silver.listings l ON l.listing_sk = r.listing_sk
  GROUP BY l.city, l.country
),
amen AS (
  SELECT
    l.city, l.country,
    COLLECT_LIST(a.bk_amenity_name) AS top_amenities
  FROM airbnb.silver.listing_amenities la
  JOIN airbnb.silver.listings   l  ON l.bk_listing_id = la.bk_listing_id
  JOIN airbnb.silver.amenities  a  ON a.bk_amenity_name = la.bk_amenity_name
  GROUP BY l.city, l.country
)
SELECT
  b.city, b.country,
  b.total_listings, b.avg_price, b.avg_rating,
  COALESCE(r.total_reviews, 0) AS total_reviews,
  COALESCE(r.active_hosts, 0)  AS active_hosts,
  a.top_amenities,
  current_timestamp() AS ingest_ts
FROM base b
LEFT JOIN rev  r ON r.city=b.city AND r.country=b.country
LEFT JOIN amen a ON a.city=b.city AND a.country=b.country;


-- -------------------------------------------
-- GOLD.AMENITY_POPULARITY
-- -------------------------------------------
INSERT OVERWRITE TABLE airbnb.gold.amenity_popularity
SELECT
  a.amenity_sk,
  l.country,
  COUNT(DISTINCT la.bk_listing_id)        AS listings_with_amenity,
  AVG(l.review_scores_rating)             AS avg_rating,
  AVG(CAST(l.price AS DOUBLE))            AS avg_price,
  current_timestamp()                     AS ingest_ts
FROM airbnb.silver.listing_amenities la
JOIN airbnb.silver.amenities a ON a.bk_amenity_name = la.bk_amenity_name
JOIN airbnb.silver.listings  l ON l.bk_listing_id   = la.bk_listing_id
GROUP BY a.amenity_sk, l.country;

### Step 2
**Objetivo:** Definir features relevantes para predecir el precio de un listing

---


In [0]:
-- Elimina datos previos
DROP TABLE IF EXISTS airbnb.gold.price_features;

-- Features para modelo de precios
CREATE TABLE airbnb.gold.price_features
USING DELTA
AS
WITH L AS (
  SELECT
    listing_sk,
    bk_listing_id,
    CAST(price AS DOUBLE)                               AS price,              -- TARGET
    CAST(accommodates AS INT)                           AS accommodates,
    CAST(bedrooms AS INT)                               AS bedrooms,
    CAST(bathrooms AS DOUBLE)                           AS bathrooms,
    CAST(beds AS INT)                                   AS beds,
    CAST(review_scores_rating AS DOUBLE)                AS review_scores_rating,
    CAST(number_of_reviews AS INT)                      AS number_of_reviews,
    
    country, city, neighborhood,
    CAST(latitude AS DOUBLE)                            AS latitude,
    CAST(longitude AS DOUBLE)                           AS longitude,
    first_review_date, last_review_date,
    host_sk
  FROM airbnb.silver.listings
),
H AS ( -- features de host
  SELECT
    host_sk,
    MAX(CAST(host_is_superhost AS BOOLEAN))             AS host_is_superhost,
    MAX(CAST(host_total_listings_count AS INT))         AS host_total_listings_count,
    MAX(CAST(host_response_rate AS INT))                AS host_response_rate
  FROM airbnb.silver.hosts
  GROUP BY host_sk
),
A AS ( -- agregados y flags de amenities por listing
  SELECT
    la.bk_listing_id,
    COUNT(*)                                           AS amenities_count,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%wifi%'                 THEN 1 ELSE 0 END) AS has_wifi,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%kitchen%'              THEN 1 ELSE 0 END) AS has_kitchen,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%washer%'               THEN 1 ELSE 0 END) AS has_washer,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%dryer%'                THEN 1 ELSE 0 END) AS has_dryer,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%heating%'              THEN 1 ELSE 0 END) AS has_heating,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%air conditioning%'     THEN 1 ELSE 0 END) AS has_air_conditioning,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%tv%'                   THEN 1 ELSE 0 END) AS has_tv,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%dishwasher%'           THEN 1 ELSE 0 END) AS has_dishwasher,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%microwave%'            THEN 1 ELSE 0 END) AS has_microwave,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%refrigerator%'         THEN 1 ELSE 0 END) AS has_refrigerator,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%stove%'                THEN 1 ELSE 0 END) AS has_stove,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%oven%'                 THEN 1 ELSE 0 END) AS has_oven,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%parking%'              THEN 1 ELSE 0 END) AS has_parking,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%pets allowed%'         THEN 1 ELSE 0 END) AS has_pets_allowed,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%smoking allowed%'      THEN 1 ELSE 0 END) AS has_smoking_allowed,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%first aid%'            THEN 1 ELSE 0 END) AS has_first_aid_kit,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%fire extinguisher%'    THEN 1 ELSE 0 END) AS has_fire_extinguisher,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%smoke detector%'       THEN 1 ELSE 0 END) AS has_smoke_detector,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%carbon monoxide%'      THEN 1 ELSE 0 END) AS has_co_detector,
    MAX(CASE WHEN a.bk_amenity_name LIKE '%waterfront%'           THEN 1 ELSE 0 END) AS is_waterfront
  FROM airbnb.silver.listing_amenities la
  JOIN airbnb.silver.amenities a
    ON a.amenity_sk = la.amenity_sk
  GROUP BY la.bk_listing_id
),
F AS ( -- ensamblado + features derivadas
  SELECT
    l.listing_sk,
    l.bk_listing_id,
    l.price,
    l.accommodates, l.bedrooms, l.bathrooms, l.beds,
    l.review_scores_rating, l.number_of_reviews,
    
    l.country, l.city, l.neighborhood,
    l.latitude, l.longitude,
    -- features temporales de reviews
    CASE
      WHEN l.first_review_date IS NULL THEN NULL
      ELSE DATEDIFF(current_date(), l.first_review_date)
    END                                               AS days_since_first_review,
    CASE
      WHEN l.last_review_date IS NULL THEN NULL
      ELSE DATEDIFF(current_date(), l.last_review_date)
    END                                               AS days_since_last_review,
    CASE
      WHEN l.first_review_date IS NULL THEN NULL
      WHEN months_between(COALESCE(l.last_review_date, current_date()), l.first_review_date) <= 0 THEN NULL
      ELSE l.number_of_reviews / months_between(COALESCE(l.last_review_date, current_date()), l.first_review_date)
    END                                               AS reviews_per_month_der,
    -- host features
    h.host_is_superhost,
    h.host_total_listings_count,
    h.host_response_rate,
    -- amenities
    COALESCE(a.amenities_count, 0)                    AS amenities_count,
    COALESCE(a.has_wifi,0)               AS has_wifi,
    COALESCE(a.has_kitchen,0)            AS has_kitchen,
    COALESCE(a.has_washer,0)             AS has_washer,
    COALESCE(a.has_dryer,0)              AS has_dryer,
    COALESCE(a.has_heating,0)            AS has_heating,
    COALESCE(a.has_air_conditioning,0)   AS has_air_conditioning,
    COALESCE(a.has_tv,0)                 AS has_tv,
    COALESCE(a.has_dishwasher,0)         AS has_dishwasher,
    COALESCE(a.has_microwave,0)          AS has_microwave,
    COALESCE(a.has_refrigerator,0)       AS has_refrigerator,
    COALESCE(a.has_stove,0)              AS has_stove,
    COALESCE(a.has_oven,0)               AS has_oven,
    COALESCE(a.has_parking,0)            AS has_parking,
    COALESCE(a.has_pets_allowed,0)       AS has_pets_allowed,
    COALESCE(a.has_smoking_allowed,0)    AS has_smoking_allowed,
    COALESCE(a.has_first_aid_kit,0)      AS has_first_aid_kit,
    COALESCE(a.has_fire_extinguisher,0)  AS has_fire_extinguisher,
    COALESCE(a.has_smoke_detector,0)     AS has_smoke_detector,
    COALESCE(a.has_co_detector,0)        AS has_co_detector,
    COALESCE(a.is_waterfront,0)          AS is_waterfront,
    current_timestamp()                  AS ingest_ts
  FROM L l
  LEFT JOIN H h       ON h.host_sk = l.host_sk
  LEFT JOIN A a       ON a.bk_listing_id = l.bk_listing_id
)
SELECT * FROM F;